# Outline
0. Discrete space setup
1. Discrete Ergodic Coverage
2. Maximizing Information Gathering

In [1]:
distribution_name = "one_peak"
time_steps = 100
K = 5
system1_name = "mm_1"
system2_name = "ig_1"
compute_mu = False
grid_length = 10

In [2]:
import numpy as np
U_shape = (1,1)
all_k = list(np.ndindex(*[K]*len(U_shape)))

# 0. Discrete Space Setup

## 0.a. Define probability distribution $\mu$

In [3]:
# defining probability distribution mu
from probability_distribution import *

mu = mu_gaussians([(np.array([0.2, 0.75]), 0.1)], U_shape)
mu = mu_normalize(mu, U_shape)

# calculating fourier coefficients of probability distribution mu
from fourier_functions import *
import pickle

ff = Fourier_Functions(mu, U_shape, K, compute_mu=compute_mu)
if compute_mu:
    mu_k = {}
    for k in ff:
        mu_k[k] = ff[k]['mu_k']

    with open(f'mu/{distribution_name}_{K}.pkl', 'wb') as handle:
        pickle.dump(mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'mu/{distribution_name}_{K}.pkl', 'rb') as handle:
        mu_k = pickle.load(handle)
    for k in ff:
        ff[k]['mu_k'] = mu_k[k]

original_mu = mu 
mu = fourier_coefficient2distribution(ff, all_k, c_k=None)
# dx = 1/grid_length
# mu_grid = np.array([[mu(np.array([c*dx, r*dx])+dx/2) for c in range(grid_length)] for r in range(grid_length)])
# def r2_to_idx(x):
#     return int(x[0]/dx), int(x[1]/dx)
# def grid_to_pdf(grid):
#     def discrete(x):
#         c,r = r2_to_idx(x)
#         return grid[r,c]
#     return discrete 
# mu = grid_to_pdf(mu_grid)

# if compute_mu:
#     mu_display2D(original_mu, U_shape, title=f"mu/{distribution_name}_original")
#     mu_display2D(mu, U_shape, title=f"mu/{distribution_name}_grid")


## 1. Generate Vanilla Ergodic Trajectory $x$

In [ ]:
from ergodic_agents import *
from mm_agent import *

agent1 = MMAgent1(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
system1 = AgentSystem([agent1], mu, U_shape, ff, K)

In [ ]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system1.evolve(t, delta_t)


In [ ]:
filename1 = f"robustness/system={system1_name}_mu={distribution_name}_K={K}_T={time_steps}"
description = f"{system1_name} on {distribution_name}"

system1.visualize_trajectory(filename1, description)
# system1.visualize2d(filename=filename1, additional_title=description, plot_c_k=False)
system1.visualize_ergodicity(filename1)

## 2. Maximize Information Gathering

In [ ]:
dx = 0.1

In [ ]:
from casadi_agent_info_gathering import *
agent2 = CasadiAgentInfoGathering(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
agent2.mu = mu
agent2.dx = dx
system2 = AgentSystem([agent2], mu, U_shape, ff, K)

In [ ]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system2.evolve(t, delta_t)

In [ ]:
filename2 = f"robustness/system={system2_name}_mu={distribution_name}_K={K}_T={time_steps}"
description = f"{system2_name} on {distribution_name}"

system2.visualize_trajectory(filename2, description)
# system2.visualize2d(filename=filename2, additional_title=description, plot_c_k=False)
system2.visualize_ergodicity(filename2)

## 3. Comparison

In [ ]:
results = {}

results['ergodic_trajectory'] = {}
results['ergodic_trajectory']['trajectory'] = agent1.x_log
results['ergodic_trajectory']['ergodicity'] = agent1.e_log
results['ergodic_trajectory']['info gathered'] = [info_gathering_metric(mu, agent1.x_log, T, 1/dx) for T in range(time_steps+1)]

results['info_gather_trajectory'] = {}
results['info_gather_trajectory']['trajectory'] = agent2.x_log
results['info_gather_trajectory']['ergodicity'] = agent2.e_log
results['info_gather_trajectory']['info gathered'] = [info_gathering_metric(mu, agent2.x_log, T, 1/dx) for T in range(time_steps+1)]
